In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

%matplotlib inline

In [7]:
! dvc pull

+-------------------------------------------+
|                                           |
|     Update available 0.66.1 -> 0.66.4     |
|       Run pip install dvc --upgrade       |
|                                           |
+-------------------------------------------+

name: ../DATA, md5: 7e5dedeab3a9b9cf7114380eea4b2dcb.dir
name: ../DATA, md5: 7e5dedeab3a9b9cf7114380eea4b2dcb.dir
Nothing to do                                                                   
ERROR: failed to pull data from the cloud - Checkout failed for following targets:
 ../DATA
Did you forget to fetch?

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!


In [3]:
DATA_FOLDER = '../DATA'
files = os.listdir(DATA_FOLDER)

FileNotFoundError: [Errno 2] No such file or directory: '../DATA'

In [4]:
data = pd.read_csv(os.path.join(
    DATA_FOLDER, '19836_sds011_sensor_.csv'), delimiter=';', parse_dates=['timestamp'], index_col=5)

FileNotFoundError: [Errno 2] File b'../DATA/19836_sds011_sensor_.csv' does not exist: b'../DATA/19836_sds011_sensor_.csv'